# Week 4 Notes
&emsp;&emsp;本周内容：ML的C4决策树和C8集成学习。这两章放到一起看还是有一定道理的，集成学习就可以理解为对多棵决策树的集成。

## ML C4 决策树
&emsp;&emsp;本书最开头的内容简介部分也提到，前三章是机器学习的基础知识，或者说一些“工具”；从第四章开始就介绍了主要的机器学习方法，也就是说，读者应从本章开始抱着具体机器学习算法思想的姿态来进行学习。而本章所讨论的决策树，则主要澳大利亚计算机科学家Ross Quinlan在推动的一种算法。
<br>&emsp;&emsp;组会时补充了决策树算法在整个ML体系当中的地位，不过我倒感觉这个cheatsheet还挺有意思的，感觉把机器学习搞得很套路了，这里附上英文和中文的scikit的cheatsheet各一份。
![](image/cheatsheet.png)
![](image/scikit-learn算法选择备忘单_修正.png)

### 4.1 基本流程
决策树是基于树结构来进行决策的，这恰是人类在面临决策问题时一种很自然的处理机制，其基本思想就是分治法。
![](image/决策树算法.png)
以上是决策树算法的一个基本流程，但实际上这些if-then都是在排除一些边边角角的特殊情况。需要注意的有以下几点：
* 决策树未必是二叉树（对应属性的取值未必只有两个）
* 第14行，某**离散**属性一旦使用，在之后的划分决策中就不再考虑该属性
* 常规情况来讲，总结一下决策树的构建就是，选择一个属性，对该属性下的几个取值做分支，“如果分的不够清楚”，就继续这样递归建树。

### 4.2 划分选择
这一部分是决策树内容中比较关键和核心的地方。对应现实中，我们人到底以一个怎样的指标进行分类，就比较见仁见智了。总的来说，我们都希望结点的“纯度”（purity）越来越高，常用以下三个指标：信息增益、增益率和基尼指数。
#### 4.2.1 信息增益
直观地理解，信息增益意味着使用该属性进行划分获得的“纯度提升”更大。而引入了“信息熵”来反映样本集合的纯度，熵越大越混乱，熵越小纯度越高。
* 首先给出信息熵的定义
$$\operatorname{Ent}(D)=-\sum_{k=1}^{|\mathcal{Y}|} p_{k} \log _{2} p_{k}$$
其中$p_k$是**第k类样本**所占的比例。这里第k类是针对分类问题的类而言的，比如二分类问题就只会有两个类。
  - 信息熵应该是从香农给出的信息量而来，需要注意的是，其最小值是0、最大值是$\log _{2} |\mathcal{Y}|$，其中$|\mathcal{Y}|$为类别总数；
  - 最小值在某一个$p_k$为1，其余$p_k$都为0时取到，这时全都是某一类，当然纯。
  - 最大值在$p_{1}=p_{2}=\ldots=p_{|\mathcal{Y}|}=\frac{1}{\log _{2} |\mathcal{Y}|}$时取到，这时每一类的样本都很平均，当然混乱。
* 再给出信息增益的定义，信息增益是关于属性的：
$$\operatorname{Gain}(D, a)=\operatorname{Ent}(D)-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \operatorname{Ent}\left(D^{v}\right)$$
  - 在总熵一定的前提下，我们自然希望某个属性的熵越小越好，也就是这个属性越纯越好
  - 属性越纯→属性熵越小→信息增益越大→越会被选择划分
* ID3决策树就是以信息在增益为准则作为划分标准的
* Case Study：西瓜数据集2.0
![](image/西瓜数据集2.0.png)
  - ①宏观想法：对每个属性计算信息增益
  - ②微观计算：对某个属性可能有k个取值（比如对“色泽”就有k=3个取值：青绿、乌黑、浅白），划分为k个子集并分别计算子集的信息熵
![](image/子集信息熵.png)
对k个子集做加权、代入式子就可以得到该属性的信息熵
  - ③取信息增益最大的属性做分支划分，递归。
<br>最终得到基于信息增益生成的决策树如下：
![](image/2.0决策树.png)
  注：像纹理模糊的那一个分支，因为都为反类，就停止继续划分了，这就是一种停止划分的“边界情况”。

#### 4.2.2 增益率
增益率（gain ratio）解决了信息增益对取值数目较多的属性的偏好，不过有一些“矫枉过正”。C4.5决策树算法就是基于增益率来选择最优划分属性，但并不是直接选择增益率最大的，而是一种启发式思想：先从候选划分属性中找出信息增益高于平均水平的，再选择增益率最高的。（why？）
* 增益率定义为：
$$\operatorname{Gain_ratio}(D, a)=\frac{\operatorname{Gain}(D, a)}{\operatorname{IV}(a)}$$
其中$\mathrm{IV}(a)$被称为属性a的“固有值”(intrinsic value):
$$\mathrm{IV}(a)=-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \log _{2} \frac{\left|D^{v}\right|}{|D|}$$
* 属性的可能取值数目越多，$\mathrm{IV}(a)$值通常会越大。

#### 4.2.3 基尼系数
基尼系数是基尼值按属性样本权重加权平均得来的。CART(Classification and Regression Tree)决策树就是使用“基尼指数”来选择属性划分的。
* 基尼值反映了从数据集中随机抽取两个样本，其类别标志不一样的概率。显然，这种概率越小，样本越纯。其定义为：
$$\begin{aligned}
\operatorname{Gini}(D) &=\sum_{k=1}^{|\mathcal{Y}|} \sum_{k^{\prime} \neq k} p_{k} p_{k^{\prime}} = \sum_{k=1}^{|\mathcal{Y}|}\left(p_{k}-p_{k}^{2}\right)\\
&=1-\sum_{k=1}^{|\mathcal{Y}|} p_{k}^{2}
\end{aligned}$$
* 基尼系数定义为：
$$\operatorname{Gini}_{-} \operatorname{index}(D, a)=\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \operatorname{Gini}\left(D^{v}\right)$$
也就是基尼值按属性权重加权平均。

### 4.3 剪枝
&emsp;&emsp;剪枝（pruning）是决策树学习算法对付“过拟合”的主要手段——事实上很多算法思想都存在“剪枝”的概念。当数据量很大时，剪枝就显得很有必要了，一些剪枝操作可以大大加快算法的运行时间。
<br>&emsp;&emsp;而剪枝又可以分为预剪枝和后剪枝。前者的思想是在建树时是否分支要做判定，判定这种分支是否在验证集上提升了精度；后者的思想则是先完整地建树，随后以验证集精度是否提升，自底向上地尝试去除，或者说融合一些分支。
* Case Study：仍是西瓜数据集2.0→但为了剪枝，这个数据集就要用留出法等方式划分出训练集和验证集。
  - 如果我们完全地建这个树，是如下图这个样子：
![](image/不剪.png)
  - 那么预剪枝就会在每一次分支做判定：若分支，分支在验证集上是否提升了精度，如下图就是对预剪枝的示意：
![](image/预剪枝.png)
  - 而后剪枝则是从完全的树自底向上地融合（也就是按照6→5→2→3→1的顺序依次判断），如下图所示
![](image/后剪枝.png)
* 预剪枝的判定标准是，分支后是否提升了验证集精度，如果没有提升就不分支（相等时考虑奥卡姆剃刀原则，也不分）
* 后剪枝的判定标准也是，去除后是否提升了验证集精度，如果没有提升就不去除（同样考虑奥卡姆剃刀原则，没有提升就减掉，更简单些）
* 预剪枝的本质是贪心，所以存在欠拟合的风险；后剪枝因为是从复杂剪到简单，泛化性能自然一般优于预剪枝。
* 但是代价就是后剪枝的训练时间开销远比预剪枝，甚至未剪枝的决策树都要大得多。

### 4.4 连续与缺失值
#### 4.4.1 连续值处理
接下来完成从离散到连续的延伸。这种转化分为两个层面，一是属性值从离散变为连续，二是如何从分类任务转换为回归任务。
* 属性连续值处理
  - 基本的思路就是，将连续的一系列属性值，用一刀切成两半；如果切得不够好，还可以再切一刀。
  - 那么这一刀切得就不能有“歧义”，于是可以选择每个连续取值的“中位点”（平均值）作为候选划分点集合
  - 于是可以对信息增益的公式进行改造：
$$\begin{aligned}
\operatorname{Gain}(D, a) &=\max _{t \in T_{a}} \operatorname{Gain}(D, a, t) \\
&=\max _{t \in T_{a}} \operatorname{Ent}(D)-\sum_{\lambda \in\{-,+\}} \frac{\left|D_{t}^{\lambda}\right|}{|D|} \operatorname{Ent}\left(D_{t}^{\lambda}\right)
\end{aligned}$$
* Case Study：西瓜数据集3.0
还是用例子来理解比较通畅一些。
![](image/西瓜3.0.png)
如图加入了密度和含糖率两个离散值。
  - 对某一属性，先进行从小到大的排序。
  - 取相邻两值的中位点做为划分候选集合（17个样本就对应16个划分点）
  - 根据上式计算出每个划分点的信息增益，取最大者作为概述性的信息增益（同时记录划分点）
  - 继续像3.0一样代入计算各个属性的信息增益，最终横向对比各属性信息增益，并选择划分属性
* 最终形成的决策树如下所示：
![](image/西瓜3.0决策树.png)
* 从分类任务转变为回归任务
  - 需要注意，回归任务的构建不能再使用信息增益、增益率、基尼系数这些指标了，而是要对应地计算方差等，让误差值指标最小来选择分类属性。
  - 最后的叶结点要取平均。比如我们的任务是预测房价，输入影响房价的各种因素，比如房屋面积、距离地铁距离等等。构建回归树之后，最后需要将叶结点的样本取平均，最终作为输出的回归结果。

#### 4.4.2 缺失值处理
现实任务中常会遇到不完整样本，比如诊测成本、隐私保护等因素。显然这时不能放弃不完整样本，常见的思路是：继续用那些不缺失的样本计算信息增益等指标，并乘以权重系数因子，以平衡各属性之间缺失样本造成的差异。
* 推广信息增益的计算式为：
$$\begin{aligned}
\operatorname{Gain}(D, a) &=\rho \times \operatorname{Gain}(\tilde{D}, a) \\
&=\rho \times\left(\operatorname{Ent}(\tilde{D})-\sum_{v=1}^{V} \tilde{r}_{v} \operatorname{Ent}\left(\tilde{D}^{v}\right)\right)
\end{aligned}$$
  - 这里的$\rho$，可以理解为一种权重系数（即将所有属性的分母都变成总样本值），也可以理解为一种“可信度”，缺失的样本越多，这种可信度自然越低。
  - 这里上波浪表示取无缺失值的样本。
  - 实际上这里的$\tilde{r}_{v}$完全可以写成$\frac{\mid \tilde{D}^{v} \mid }{\mid \tilde{D} \mid}$，整个计算过程和之前是非常类似的。
* 选择了划分的属性后，对缺失样本，需要将其以对应的权重同时划分到多个属性中。比如有7个样本进入了“纹理=清晰”，有5个样本进入了“纹理=稍糊”分支，有3个样本进入了“纹理=模糊”分支，此时有一个样本缺失，那么这个样本就要以$\frac{7}{15}$、$\frac{5}{15}$、$\frac{3}{15}$的权重对应进入这些分支中
* 但是缺失样本在计算信息增益准备分支的阶段是不做考虑的，这一点需要注意

### 4.5 多变量决策树
此节是从多维空间的角度去考虑建树任务的。d个属性描述的样本点就对应了d维空间的一个数据点。比如我们考虑两个属性（含糖率、密度），实际上这就张成了一个二维平面。对应地，图中的红线就对应了决策树的分支判断。
![](image/单变量.png)
* 如果每次划分只考虑一个变量，那么这种划分就会是平行于坐标轴的，会切得“方方正正的”。
* 而如果考虑多个变量的线性模型，这种切割就会更加“柔滑”，反应在空间中就是一个个超平面。
![](image/多变量.png)

## ML C8 集成学习
&emsp;&emsp;集成学习是指通过**构建并结合多个学习器**来完成学习任务。其一般结构如下图所示：
![](image/集成学习.png)
&emsp;&emsp;这些个体学习器通常由一个现有的学习算法从训练数据中产生，比如决策树算法、BP神经网络算法等。
<br>&emsp;&emsp;集成的思想主要可以分为两大类，一是个体学习器间存在强依赖关系、必须串行生成的序列化方法，其代表是Boosting：将弱学习器提升为强学习器；另一类是不存在强依赖关系、可同时生成的并行化方法，代表是Bagging与随机森林，即希望这些弱学习器能相对独立，并将他们通过加权、投票等方式最终共同输出结论（实际上Boosting最后也需要一些方式让这些学习器共同输出结论，所以Boosting在泛化性能上往往更佳）。

### 8.2 Boosting
Boosting族最著名的代表是AdaBoost（Adaptive Boosting），关于AdaBoost目前解释的比较好的文章可以参见https://zhuanlan.zhihu.com/p/41536315 ，结合了具体的实例就很好理解AdaBoost的思想了。其中也附了几个链接具体讨论AdaBoost的公式推导等，在此不多做展开了。
<br>总之AdaBoost的核心思想就是不断地迭代，将那些误差较大的样本“凸显出来”，然后不断地针对它做优化。

### 8.3 Bagging与随机森林
#### 8.3.1 Bagging
Bagging的缩写来源于Bootstrap AGGregatING，所以它是基于自助采样法的。
* Bagging的基本流程如下：
<br>1.先采样出T个含m个训练样本的采样集
<br>2.基于每个采样集训练出一个基学习器
<br>3.再将这些学习器进行结合，通常对分类任务使用简单投票法，对回归任务使用简单平均法
* 假定基学习器的计算复杂度是O(m)，那么Bagging的复杂度大致为T(O(m)+O(s))，T通常是个不太大的常数，所以Bagging集成与直接使用基学习器的复杂度同阶，所以Bagging是很高效的。
* AdaBoost只适用于二分类任务（不过目前已有对于多分类和回归任务的变体），Bagging可以不经修改地用于多分类、回归任务。
* 自主采样过程为基学习器带来了包外估计，算是Bootstrapping的一些优点也继承了过来，

#### 8.3.2 随机森林
随机森林（Random Forest，简称RF）是Bagging的一个变体。形象地说，就是一颗颗决策树构成的森林。故RF以决策树为基学习器，在其训练过程中引入了随机属性选择。传统决策树是在当前结点的属性集合中选择一个最优属性，而RF则是先从中随机选择k个属性的自己，再选择最优者做划分。这里的k控制了随机性的引入程度：
1.k=1意味着完全地就是随机选择一个属性用于划分
2.k=d（属性总数）意味着与传统决策树相同
3.通常，推荐$k=log_2 d$。
* 随机森林简单、容易实现、计算开销小，并且在很多现实任务中展现出强大的性能。
* 它的关键作用在于增加了基学习器的多样性，不仅来自样本扰动，还来自属性扰动，这就使得最终集成的泛化性能可以通过个体学习器之间的差异度增加进一步提升。
* 随机森林的训练效率常优于Bagging，这很好理解，因为Bagging是使用了全部样本，而RF则是随机了一部分样本，自然开销更小。
***
**总结：**从偏差-方差分解的角度看，Boosting主要关注降低误差，Bagging和RF主要关注降低方差，因此Boosting可以基于泛化性能相当弱的学习器构建出很强的集成，Bagging则在不剪枝决策树、神经网络等易受样本扰动的学习器上效用更为明显。训练效率上，RF＞Bagging＞Boosting，而在工业界泛化能力通常认为Boosting更强。

## 8.4 结合策略
本节内容可以很好地以结构框图来表示。
* 学习器结合可能带来的好处
![](image/结合的好处.png)
* 结合的方法
![](image/结合的方法.png)
* Stacking是学习法的典型代表，其思想是将初级学习器的输出，和样本本来的标记共同构成新样本，用于训练次级学习器
* 若直接使用初级学习器的训练集来产生次级训练集，则过拟合风险比较大，所以一般是使用交叉验证法或留一法